**Step 1: Setup TensorFlow 1.x**

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.2


**Step 2: Clone the tensorflow-yolo-v3 repository**

In [ ]:
# Clone
!git clone https://github.com/mystic123/tensorflow-yolo-v3

Cloning into 'tensorflow-yolo-v3'...
remote: Enumerating objects: 219, done.
remote: Total 219 (delta 0), reused 0 (delta 0), pack-reused 219
Receiving objects: 100% (219/219), 57.63 KiB | 8.23 MiB/s, done.
Resolving deltas: 100% (112/112), done.


**Step 3a: Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 2016-04-24
'3D Augmented Reality - ProjectA1-ElifSEVEN      .gslides'
 3DProject-A1-ElifSeven_1230695.gdoc
'Adsız doküman (1).gdoc'
'Adsız doküman.gdoc'
'Adsız sunu.gslides'
 AllSbloccoContratto.pdf
 backup
 BitirmeProje
 BitirmeProje1.rar
'Colab Notebooks'
'COURSE TRACK CHANGE FORM.docx'
'Delega ISEE parificato_Authorization form.pdf'
 demo1ToDoList.zip
'Elif Seven_CV.gdoc'
'Elif Seven_CV.pdf'
 ELİFSEVEN-CV.pdf
'Elif Seven_CV_updated-2 (1).pdf'
'Elif Seven_CV_updated-2.pdf'
 ELİFSEVEN-CV-UPDATE.pdf
'ELİF SEVEN.gdoc'
'Epinions Social Network Analysis.gdoc'
 Evraklar
 flare0.jpg
 flare-oil-smoke-dataset.zip
 flaring-against-blue-sky_1600.jpg
'Huawei Demo Task SUNUM.gslides'
'Huawei Demo Task SUNUM.pdf'
'İngilizce Temel Gramer.pdf'
'İş Görüşmeleri.gdoc'
 italya-doc
'Meeting Topics - 01.gdoc'
 MeetingTopics.gdoc
'Mektup .gdoc'
'My Drive'
'Network Science Homework1 & Homework2 Presentation.gslides'
 NeuralNetworkHW1-ElifSeven.gdoc
 NeuralNetworkHW3-El

**Step 3b: Navigate to tensorflow-yolo-v3 repository**

In [ ]:
%cd tensorflow-yolo-v3/

/content/tensorflow-yolo-v3


**Step 4: Download .weights and .classes files**

In [ ]:
!gdown --id <1h3WuNr1yZzdlRam59fN8cawHqtM_avEl>  #classes.txt
!gdown --id <1BI4xoHzsgbf8HGZv9kEtIVzQCML-3ZXx>  #yolov3_training_last.weights

/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `gdown --id <1h3WuNr1yZzdlRam59fN8cawHqtM_avEl>  #classes.txt'
/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `gdown --id <1BI4xoHzsgbf8HGZv9kEtIVzQCML-3ZXx>  #yolov3_training_last.weights'


**Step 5: Convert YOLOv3 .weights to .pb**

In [ ]:
!python convert_weights_pb.py --class_names "/mydrive/yolov3-new/final-rpject-yolo/tensorflowLite/classes.txt" --weights_file "/mydrive/yolov3-new/final-rpject-yolo/tensorflowLite/yolov3-custom_last .weights" --data_format "NHWC" 

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0220 18:11:02.139028 139719807031168 module_wrapper.py:139] From convert_weights_pb.py:42: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0220 18:11:02.140459 139719807031168 module_wrapper.py:139] From convert_weights_pb.py:44: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

Instructions for updating:
Please use `layer.__call__` method instead.
W0220 18:11:02.143505 139719807031168 deprecation.py:323] From /tensorflow-1.15.2/python3.6/tensorflow_core/contrib/layers/python/layers/layers.py:1057: Layer.apply (from tensorflow.python.keras.engine.

**Step 6a: Restart runtime**

Click on Runtime on the menu bar and select **Restart runtime.**

**Step 6b: Setup TensorFlow 2.x**

In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [ ]:
import tempfile
import os
import glob

**Step 7a: Navigate to tensorflow-yolo-v3 repository**

In [ ]:
%cd tensorflow-yolo-v3/

/content/tensorflow-yolo-v3


**Step 7b: Check input and output nodes of neural network**


In [ ]:
import tensorflow.compat.v1 as tf
gf = tf.GraphDef()   
m_file = open('frozen_darknet_yolov3_model.pb','rb')
gf.ParseFromString(m_file.read())

with open('somefile.txt', 'a') as the_file:
     for n in gf.node:
         the_file.write(n.name+'\n')

file = open('somefile.txt','r')
data = file.readlines()
output = data[len(data)-1]
print("Output array = ", output)

file.seek ( 0 )
input=file.readline()
print("Input array = ", input)

Output array =  output_boxes

Input array =  inputs



**Step 8: Convert frozen_darknet_yolov3_model.pb to .tflite**

In [ ]:
# Reference: https://github.com/sayakpaul/Adventures-in-TensorFlow-Lite/blob/master/DeepLabV3/DeepLab_TFLite_COCO.ipynb
# Load the TensorFlow model
# The preprocessing and the post-processing steps should not be included in the TF Lite model graph 
# because some operations (ArgMax) might not support the delegates. 
# Insepct the graph using Netron https://lutzroeder.github.io/netron/
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='/content/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb', 
    input_arrays = ['inputs'],   # Here, 'inputs' is the value of input array from Step 7b
    output_arrays = ['output_boxes'], # Here, 'output_boxes' is the value of output array from Step 7b
    input_shapes={'inputs': [1, 416, 416, 3]} # Here, 'inputs' is the value of input array from Step 7b
)

# Optional: Perform the simplest optimization known as post-training dynamic range quantization.
# https://www.tensorflow.org/lite/performance/post_training_quantization#dynamic_range_quantization
# You can refer to the same document for other types of optimizations.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert to TFLite Model
tflite_model = converter.convert()

_, dynamic_tflite_path = tempfile.mkstemp('.tflite')
tflite_model_size = open(dynamic_tflite_path, 'wb').write(tflite_model)
tf_model_size = os.path.getsize('/content/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb')
print('TensorFlow Model is  {} bytes'.format(tf_model_size))
print('TFLite Model is      {} bytes'.format(tflite_model_size))
print('Post training dynamic range quantization saves {} bytes'.format(tf_model_size-tflite_model_size))

TensorFlow Model is  246555459 bytes
TFLite Model is      62362928 bytes
Post training dynamic range quantization saves 184192531 bytes


In [ ]:
!ls -lh {dynamic_tflite_path}

-rw------- 1 root root 60M Feb 20 18:12 /tmp/tmp9cjmzvb4.tflite


**Step 9: Move the .tflite model into your Google Drive folder**

In [ ]:
tflite_file = ''
tflite_file = tflite_file.join(glob.glob("/tmp/*.tflite")) # Find the temp file of type .tflite
tflite_file = tflite_file[5:len(tflite_file)]  # Extract .tflite file name.

%cd ..
%cd ../tmp
os.rename(tflite_file, 'detect.tflite') # Rename the temp .tflite to detect.tflite
%mv detect.tflite ../content/tensorflow-yolo-v3//  # Move detect.tflite to Goolge Drive

/
/tmp


**Step 10: Model Inference**

In [ ]:
interpreter = tf.lite.Interpreter(model_path='../content/tensorflow-yolo-v3/detect.tflite')

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
input_details

[{'dtype': numpy.float32,
  'index': 0,
  'name': 'inputs',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 416, 416,   3], dtype=int32),
  'shape_signature': array([  1, 416, 416,   3], dtype=int32),
  'sparsity_parameters': {}}]

In [ ]:
output_details

[{'dtype': numpy.float32,
  'index': 407,
  'name': 'output_boxes',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([    1, 10647,     9], dtype=int32),
  'shape_signature': array([    1, 10647,     9], dtype=int32),
  'sparsity_parameters': {}}]

**Step 11: Navigate to Google Drive**

In [ ]:
%cd /content/gdrive/MyDrive/yolov3-new/final-rpject-yolo/tensorflowLite/test_images

/content/gdrive/MyDrive/yolov3-new/final-rpject-yolo/tensorflowLite/test_images


In [ ]:
pip install tensorflow==2.3.1


     |████████████████████████████████| 320.4MB 54kB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 460kB 58.3MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
pip install opencv-python


In [ ]:
pip install opencv-contrib-python


In [ ]:
pip install pillow


In [ ]:
pip install numpy


**Step 12: Test the model**

In [ ]:
!python /content/gdrive/MyDrive/yolov3-new/final-rpject-yolo/tensorflowLite/test_images/test_tflite.py

2021-02-20 19:38:49.173092: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/yolov3-new/final-rpject-yolo/tensorflowLite/test_images/test_tflite.py", line 138, in <module>
    interpreter = tf.lite.Interpreter(model_path=model_path)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/lite/python/interpreter.py", line 198, in __init__
    model_path, self._custom_op_registerers))
ValueError: Didn't find op for builtin opcode 'CONV_2D' version '5'
Registration failed.

